In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MT']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('microtubule', 1378),
 ('metallothionein', 1243),
 ('melatonin', 188),
 ('magnetization transfer', 95),
 ('methyltestosterone', 64),
 ('mutant', 59),
 ('movement time', 37),
 ('mechanical thrombectomy', 36),
 ('methyltransferase', 29),
 ('middle t antigen', 26),
 ('myogenic tone', 22),
 ('massive transfusion', 21),
 ('metacyclic trypomastigotes', 20),
 ('microbial translocation', 19),
 ('moderate training', 19),
 ('motor threshold', 18),
 ('mitochondria', 17),
 ('massage therapy', 17),
 ('mental toughness', 16),
 ('methanethiol', 16),
 ('mammary tumor', 14),
 ('milk thistle', 13),
 ('middle t', 13),
 ('mechanotransducer', 13),
 ('matrine', 12),
 ('muscle thickness', 12),
 ('mindfulness training', 12),
 ('mesotocin', 12),
 ('middle temporal area', 12),
 ('mycobacterium tuberculosis', 11),
 ('manual therapy', 11),
 ('metallothionein i', 11),
 ('mirror therapy', 10),
 ('mannitol treated', 9),
 ('middle temporal', 9),
 ('malpighian tubules', 9),
 ('mechanical trauma', 9),
 ('mouth throat

In [8]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [9]:
names

{'GO:GO:0005874': 'microtubule',
 'FPLX:Metallothionein': 'Metallothionein',
 'CHEBI:CHEBI:16796': 'melatonin',
 'CHEBI:CHEBI:27436': 'methyltestosterone',
 'MESH:D013995': 'Time',
 'MESH:D017131': 'Thrombectomy',
 'CHEBI:CHEBI:16981': 'O-(3-O-D-galactosyl-N-acetyl-beta-D-galactosaminyl)-L-serine',
 'EFO:0004024': 'translocation',
 'GO:GO:0005739': 'mitochondrion',
 'MESH:D013812': 'Therapeutics',
 'CHEBI:CHEBI:16007': 'methanethiol',
 'MESH:D001943': 'Breast Neoplasms',
 'MESH:D020944': 'Milk Thistle',
 'CHEBI:CHEBI:16857': 'L-threonine',
 'CHEBI:CHEBI:6700': 'Matrine',
 'MESH:C001713': 'mesotocin',
 'EFO:0001696': 'area',
 'MESH:D009169': 'Mycobacterium tuberculosis',
 'MESH:D008317': 'Malpighian Tubules',
 'MESH:D014947': 'Wounds and Injuries',
 'MESH:D010614': 'Pharynx',
 'HGNC:6278': 'KCNK3',
 'MESH:D009154': 'Mutation',
 'HGNC:15860': 'PRPF6',
 'MESH:D014437': 'Typhus, Endemic Flea-Borne',
 'MESH:D050259': 'Genes, Mitochondrial',
 'MESH:D044094': 'Receptors, Melatonin',
 'CHEBI:C

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-10 03:54:32] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'magnetisation transfer': 'ungrounded',
  'magnetization target': 'ungrounded',
  'magnetization transfer': 'ungrounded',
  'magnetization tweezers': 'ungrounded',
  'maintenance treatment': 'ungrounded',
  'malathion': 'CHEBI:CHEBI:6651',
  'malpighian tubules': 'MESH:D008317',
  'mammary tumor': 'MESH:D001943',
  'mammillary layer thickness': 'ungrounded',
  'mannitol': 'CHEBI:CHEBI:29864',
  'mannitol treated': 'ungrounded',
  'manual therapy': 'ungrounded',
  'manual thrombectomy': 'MESH:D017131',
  'massage therapy': 'ungrounded',
  'massive transfusion': 'ungrounded',
  'masson s trichrome': 'ungrounded',
  'matrine': 'CHEBI:CHEBI:6700',
  'mechanical threshold': 'ungrounded',
  'mechanical thrombectomy': 'MESH:D017131',
  'mechanical trauma': 'MESH:D014947',
  'mechanotransducer': 'ungrounded',
  'media thickness': 'ungrounded',
  'medial thalamus': 'ungrounded',
  'medical therapy': 'ungrounded',
  'medical treatment': 'ungrounded',
  'melatonin': 'CHEBI:CHEBI:16796',
  'mela

In [35]:
grounding_map, names, pos_labels = [{'magnetisation transfer': 'ungrounded',
  'magnetization target': 'ungrounded',
  'magnetization transfer': 'ungrounded',
  'magnetization tweezers': 'ungrounded',
  'maintenance treatment': 'ungrounded',
  'malathion': 'CHEBI:CHEBI:6651',
  'malpighian tubules': 'MESH:D008317',
  'mammary tumor': 'MESH:D001943',
  'mammillary layer thickness': 'ungrounded',
  'mannitol': 'CHEBI:CHEBI:29864',
  'mannitol treated': 'ungrounded',
  'manual therapy': 'ungrounded',
  'manual thrombectomy': 'MESH:D017131',
  'massage therapy': 'ungrounded',
  'massive transfusion': 'ungrounded',
  'masson s trichrome': 'ungrounded',
  'matrine': 'CHEBI:CHEBI:6700',
  'mechanical threshold': 'ungrounded',
  'mechanical thrombectomy': 'MESH:D017131',
  'mechanical trauma': 'MESH:D014947',
  'mechanotransducer': 'ungrounded',
  'media thickness': 'ungrounded',
  'medial thalamus': 'ungrounded',
  'medical therapy': 'ungrounded',
  'medical treatment': 'ungrounded',
  'melatonin': 'CHEBI:CHEBI:16796',
  'melatonin receptor': 'MESH:D044094',
  'melt type': 'ungrounded',
  'membrane type': 'ungrounded',
  'mental task': 'ungrounded',
  'mental toughness': 'ungrounded',
  'mesenchymal transition': 'ungrounded',
  'mesenchymal type': 'ungrounded',
  'mesotocin': 'MESH:C001713',
  'met allothionein': 'ungrounded',
  'metacyclic trypomastigotes': 'ungrounded',
  'metal lothionein': 'ungrounded',
  'metal thickness': 'ungrounded',
  'metallothionein': 'FPLX:Metallothionein',
  'metallothionein i': 'FPLX:Metallothionein',
  'metastatic tumor': 'EFO:0009709',
  'metformin': 'CHEBI:CHEBI:6801',
  'methanethiol': 'ungrounded',
  'methanosaetaceae': 'ungrounded',
  'methiothepin': 'CHEBI:CHEBI:64203',
  'methoxytryptamine': 'CHEBI:CHEBI:2089',
  'methyl dl tryptophan': 'ungrounded',
  'methyl testosterone': 'NCIT:C648',
  'methyl thiophanate': 'CHEBI:CHEBI:35014',
  'methyl transferase': 'ungrounded',
  'methyltestosterone': 'NCIT:C648',
  'methylthioninium': 'ungrounded',
  'methyltransferase': 'MESH:D008780',
  'metyrapone': 'CHEBI:CHEBI:44241',
  'micro tom': 'ungrounded',
  'microbial translocation': 'ungrounded',
  'microtom': 'ungrounded',
  'microtubule': 'MESH:D008870',
  'middle t': 'MESH:D000952',
  'middle t antigen': 'MESH:D000952',
  'middle temporal': 'ungrounded',
  'middle temporal area': 'ungrounded',
  'middle temporal visual area': 'ungrounded',
  'middle turbinate': 'ungrounded',
  'milk thistle': 'ungrounded',
  'miltefosine transport': 'ungrounded',
  'mindfulness training': 'ungrounded',
  'minimum threshold': 'ungrounded',
  'mirror therapy': 'ungrounded',
  'mito tempo': 'ungrounded',
  'mito tempol': 'ungrounded',
  'mitochondria': 'ungrounded',
  'mitochondrial': 'ungrounded',
  'mitochondrial toxicity': 'ungrounded',
  'mitral tufted': 'ungrounded',
  'mixed type': 'ungrounded',
  'modalities transition': 'ungrounded',
  'moderate training': 'ungrounded',
  'monoterpenes': 'MESH:D039821',
  'monotherapy': 'ungrounded',
  'montelukast': 'CHEBI:CHEBI:50730',
  'morphological transformation': 'ungrounded',
  'motor threshold': 'ungrounded',
  'motor training': 'ungrounded',
  'mouth throat': 'ungrounded',
  'movement time': 'ungrounded',
  'mt': 'FPLX:Metallothionein',
  'mucosal thickening': 'ungrounded',
  'mucosal thickness': 'ungrounded',
  'multiple trauma': 'MESH:D009104',
  'murine typhus': 'MESH:D014437',
  'muscle thickness': 'ungrounded',
  'mushroom tyrosinase': 'ungrounded',
  'music therapy': 'MESH:D009147',
  'mutant': 'ungrounded',
  'mutant mice': 'ungrounded',
  'mutant type': 'ungrounded',
  'mutarotase': 'ungrounded',
  'mutations': 'MESH:D009154',
  'mycobacterium tuberculosis': 'MESH:D009169',
  'myogenic tone': 'ungrounded',
  'myotubes': 'MESH:D018485',
  'treatment with mt 63 78': 'ungrounded'},
 {'CHEBI:CHEBI:6651': 'malathion',
  'MESH:D008317': 'Malpighian Tubules',
  'MESH:D001943': 'Breast Neoplasms',
  'CHEBI:CHEBI:29864': 'mannitol',
  'MESH:D017131': 'Thrombectomy',
  'CHEBI:CHEBI:6700': 'Matrine',
  'MESH:D014947': 'Wounds and Injuries',
  'CHEBI:CHEBI:16796': 'melatonin',
  'MESH:D044094': 'Receptors, Melatonin',
  'MESH:C001713': 'mesotocin',
  'FPLX:Metallothionein': 'Metallothionein',
  'EFO:0009709': 'metastatic neoplasm',
  'CHEBI:CHEBI:6801': 'metformin',
  'CHEBI:CHEBI:64203': 'methiothepin',
  'CHEBI:CHEBI:2089': 'O-methylserotonin',
  'NCIT:C648': 'Methyltestosterone',
  'CHEBI:CHEBI:35014': 'thiophanate-methyl',
  'MESH:D008780': 'Methyltransferases',
  'CHEBI:CHEBI:44241': 'metyrapone',
  'MESH:D008870': 'Microtubules',
  'MESH:D000952': 'Antigens, Polyomavirus Transforming',
  'MESH:D039821': 'Monoterpenes',
  'CHEBI:CHEBI:50730': 'montelukast',
  'MESH:D009104': 'Multiple Trauma',
  'MESH:D014437': 'Typhus, Endemic Flea-Borne',
  'MESH:D009147': 'Music Therapy',
  'MESH:D009154': 'Mutation',
  'MESH:D009169': 'Mycobacterium tuberculosis',
  'MESH:D018485': 'Muscle Fibers, Skeletal'},
 ['CHEBI:CHEBI:16796', 'FPLX:Metallothionein', 'MESH:D008870', 'MESH:D008780', 'MESH:D000952', 'NCIT:C648']]

In [36]:
excluded_longforms = []

In [37]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [38]:
additional_entities = {}

In [39]:
unambiguous_agent_texts = {}

In [40]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [41]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 04:22:22] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 04:24:27] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9348503047427027 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [31]:
classifier.stats

{'label_distribution': {'MESH:D008870': 1115,
  'CHEBI:CHEBI:6651': 3,
  'FPLX:Metallothionein': 1026,
  'CHEBI:CHEBI:16796': 130,
  'MESH:D000952': 33,
  'MESH:C001713': 12,
  'MESH:D009169': 8,
  'MESH:D008780': 23,
  'MESH:D001943': 8,
  'ungrounded': 440,
  'NCIT:C648': 55,
  'CHEBI:CHEBI:29864': 2,
  'CHEBI:CHEBI:64203': 2,
  'MESH:D017131': 32,
  'MESH:D009104': 2,
  'MESH:D018485': 3,
  'MESH:D014947': 5,
  'CHEBI:CHEBI:2089': 4,
  'MESH:D009147': 6,
  'MESH:D008317': 7,
  'CHEBI:CHEBI:35014': 3,
  'CHEBI:CHEBI:50730': 2,
  'CHEBI:CHEBI:44241': 1,
  'MESH:D039821': 2,
  'CHEBI:CHEBI:6700': 9,
  'MESH:D014437': 3,
  'MESH:D044094': 5,
  'MESH:D009154': 5,
  'CHEBI:CHEBI:6801': 3,
  'EFO:0009709': 1},
 'f1': {'mean': 0.934844, 'std': 0.012562},
 'precision': {'mean': 0.925083, 'std': 0.012848},
 'recall': {'mean': 0.947193, 'std': 0.012646},
 'MESH:D009154': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'FPLX:Metallothion

In [32]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [33]:
disamb.dump(model_name, results_path)

In [34]:
print(disamb.info())

Disambiguation model for MT

Produces the disambiguations:
	Antigens, Polyomavirus Transforming	MESH:D000952
	Breast Neoplasms	MESH:D001943
	Malpighian Tubules	MESH:D008317
	Matrine	CHEBI:CHEBI:6700
	Metallothionein*	FPLX:Metallothionein
	Methyltestosterone*	NCIT:C648
	Methyltransferases*	MESH:D008780
	Microtubules*	MESH:D008870
	Monoterpenes	MESH:D039821
	Multiple Trauma	MESH:D009104
	Muscle Fibers, Skeletal	MESH:D018485
	Music Therapy	MESH:D009147
	Mutation	MESH:D009154
	Mycobacterium tuberculosis	MESH:D009169
	O-methylserotonin	CHEBI:CHEBI:2089
	Receptors, Melatonin	MESH:D044094
	Thrombectomy	MESH:D017131
	Typhus, Endemic Flea-Borne	MESH:D014437
	Wounds and Injuries	MESH:D014947
	malathion	CHEBI:CHEBI:6651
	mannitol	CHEBI:CHEBI:29864
	melatonin*	CHEBI:CHEBI:16796
	mesotocin	MESH:C001713
	metastatic neoplasm	EFO:0009709
	metformin	CHEBI:CHEBI:6801
	methiothepin	CHEBI:CHEBI:64203
	metyrapone	CHEBI:CHEBI:44241
	montelukast	CHEBI:CHEBI:50730
	thiophanate-methyl	CHEBI:CHEBI:35014

Class 

In [29]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1